In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 1

import numpy as np
from scipy.integrate import ode

import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

%aimport lorenz

# The Kalman filter

The Kalman filter is a procedure to estimate the state of a system with known dynamics, but which can only be observed imperfectly.

Consider everyone's favorite example, the Lorenz attractor:

$\dot x = \sigma(y - x)$

$\dot y = x(\rho - z) - y$

$\dot z = xy - \beta z$

Given an initial condition $[x_0, y_0, z_0]$, we can use our favorite ODE solver to compute the trajectory of this initial condition.

In [ ]:
q = lorenz.lorenz([5.0, 5.0, 5.0], 0.0, 10.0)

fig = plt.figure()
ax = fig.gca(projection='3d')
ax.plot(q[:,0], q[:,1], q[:,2])

plt.show()

What if we only get to observe the system at discrete time intervals?

What if we don't even get to observe the system, just the output of some sensor?

And what if the sensor makes errors?

In [ ]:
d = lorenz.observational_data

fig = plt.figure()
ax = fig.gca(projection='3d')
ax.scatter(d[:,0], d[:,1], d[:,2], color='r', marker='.')

plt.show()

How can we use the (admittedly noisy) observations together with what we know about the underlying system?

# Some stats

There's a random variable $x$ we'd like to estimate; we already have an unbiased estimate $\hat x_0$, that is,

$$ \hat x_0 = x + \xi, $$

where $\xi \sim N(0, C)$ for some covariance matrix $C$.
Never mind where $\hat x_0$ came from for now, we'll get to that.

Now we get a measurement $y$ of $x$:

$$ y = Mx + \eta $$

where $\eta \sim N(0, Q)$.
Updated estimate $\hat x$ has the maximum a posteriori probability:

$$ \hat x = \hat x_0 + CM^*(MCM^* + Q)^{-1}(y - M\hat x_0) $$

The new estimate has

$$ \textrm{cov}(\hat x) = C - CM^*(MCM^* + Q)^{-1}MC. $$

Observe that $\textrm{cov}(\hat x) \le \textrm{cov}(\hat x_0)$.
(Derivations on the board if you want to see them.)

In [ ]:
%aimport mle
nn = 16

# True vector to be estimated
z = np.array(map(lambda k: np.sin(2 * np.pi * k / nn), range(nn)))

L = np.zeros((nn, nn))
for i in range(nn):
    j = (i + 1) % nn
    L[i, i] = 1.0
    L[i, j] = 0.5

sigma_process = 0.25
    
# A priori estimate and covariance
x0 = z + np.dot(L, np.random.randn(nn)) * sigma_process
C0 = np.dot(L, L.T) * sigma_process**2

# We only get to measure every other point
M = np.zeros((nn/2, nn))
for i in range(nn):
    k = i/2
    M[k, i] = 1.0

# The measurement errors are white
sigma_measurement = 0.25
Q = np.eye(nn/2) * sigma_measurement**2

plt.figure()

# Plot the a priori guess in blue
plt.plot(range(nn), x0, 'b--', hold = True, linewidth=2.0)

# Artificially generate some measurements
num_trials = 32
for t in range(num_trials):
    y = np.dot(M, z) + np.random.randn(nn/2) * sigma_measurement
    x, C = mle.update(x0, C0, y, M, Q)

    cl = (num_trials - 1.0 * t) / num_trials
    plt.plot(range(nn), x, color = (cl, cl, cl), hold = True)

    x0 = x
    C0 = C

# Plot the true vector in red
plt.plot(range(nn), z, 'r--', hold = True, linewidth=2.0)
plt.show()

# Linear stochastic dynamical systems

Consider a discrete dynamical system

$$X_k = AX_{k - 1} + \xi_k.$$

$\xi_k$ is the *process noise*; $\xi_k \sim N(0, C)$.

However, we don't get to observe the process $X_k$ directly; rather, we only get to see

$$Y_k = MX_k + \eta_k,$$

where $\eta_k \sim N(0, Q)$ is the *measurement noise*.

### Prediction

Say we have an unbiased estimate $\hat x_{k-1|k-1}$ for $x_{k-1}$ based on the previous $k - 1$ observations, for which we somehow know that

$$ \textrm{cov}(\hat x_{k-1|k-1}) = P_{k|k-1}. $$

How can we update this estimate for the next timestep?

$$\hat x_{k|k-1} \equiv A\hat x_{k-1|k-1}$$

is unbiased for $x_k$.
What happens to the covariance?

$$ P_{k|k-1} \equiv \textrm{cov}(\hat x_{k|k-1}) = AP_{k-1|k-1}A^* + C$$

### Correction

Now we get some data $y_k$; how do we correct our prediction?
Well, $\hat x_{k|k-1}$ is an unbiased estimate for $x_k$, and presumably we know its covariance.
But we just did that a second ago; the a prior covariance $C$ is $P_{k|k-1}$:

$$ \hat x_{k|k} = \hat x_{k|k-1} + K_k(y_k - M\hat x_{k|k-1}) $$

where $K_k$ is the Kalman gain matrix:

$$ K_k = P_{k|k-1}M^*(MP_{k|k-1}M^* + Q)^{-1}$$

With the updated covariance:

$$ P_{k|k} = (I - K_kM)P_{k|k-1}$$

we can start over again at step $k$.

#### Appease the numerical analysts

All this can be done by keeping the Cholesky decomposition of the covariance matrix and not the inverse.

# A simple demo

Let's try this for a discrete-time approximation of the underdamped harmonic oscillator

$$ \ddot x + \gamma\dot x + \omega^2 x = \sigma^2\xi_t.$$

Letting $p = \dot x$, this is a 2D, first-order dynamical system.
To make things interesting, let's assume we can only measure $x$ but not $p$.

In [ ]:
dt = 0.01
frequency = 1.0
damping_time = 10.0

omega = 2 * np.pi / frequency
gamma = 1.0 / damping_time
A = np.linalg.inv(np.eye(2) - dt * np.array([[0.0, 1.0], [-omega**2, -gamma]]))

process_noise = 3.0
C = np.array([[0.0, 0.0],
              [0.0, process_noise**2]])

x0 = np.array([1.0, 0.0])

num_cycles = 10
num_timesteps = int(frequency / dt * num_cycles)

# We can directly measure the oscillator position, but not its velocity.
# The measurement matrix is not of full rank.
M = np.array([[1.0, 0.0], [0.0, 0.0]])
measurement_noise = 0.1
Q = np.diag([measurement_noise**2, 10 * measurement_noise**2])

x = np.zeros((num_timesteps + 1, 2))
xp = np.zeros((num_timesteps + 1, 2))
Cp = np.zeros((num_timesteps + 1, 2, 2))
x[0, :] = x0
xp[0, :] = x0
Cp[0, :, :] = C
for n in range(num_timesteps):
    x[n + 1, :] = np.dot(A, x[n, :]) + dt * np.dot(C, np.random.randn(2))
    y = np.dot(M, x[n + 1, :]) + np.dot(Q, np.random.randn(2))
    
    x_guess = np.dot(A, xp[n, :])
    C_guess = np.dot(A, np.dot(Cp[n,:], A.T)) + C
    
    xp[n+1,:], Cp[n+1,:,:] = mle.update(x_guess, C_guess, y, M, Q)

plt.figure()
plt.plot(x[:,0], x[:,1], color = 'b', hold = True)
plt.plot(xp[:,0], xp[:,1], color = 'r')


Try switching which field we can observe (velocity instead of position) or changing the noise amplitude and see how well we can follow the trajectory.

# Generalizations

* We could be estimating $x_k$ based on the entire time series of measurements, including measurements that occurred before and after $k$ (Kalman smoothing, hindcasting).
* There can be deterministic forcing:
$$ x_k = Ax_{k-1} + Bu_k + \xi_k $$
* The underlying dynamical system can be continuous (Kalman-Bucy filter, hybrid filtering):
$$ \dot x = Ax + \xi_t $$
* The underlying dynamical system need not be linear:
$$ \dot x = f(x) + \xi_t $$
Use the linearization of $f$ about the a priori estimate (extended Kalman filter).
* The system can be large (think weather forecasting), in which case maintaining covariance matrices is expensive.
Then we use an *ensemble* of representative solutions to estimate the true covariance (ensemble Kalman filter).